In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)

In [1]:
!nvidia-smi

Fri May 21 02:25:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Init

In [3]:
!pip install transformers==4.5.0

     |████████████████████████████████| 2.2MB 6.7MB/s 
     |████████████████████████████████| 3.3MB 28.7MB/s 
     |████████████████████████████████| 901kB 53.1MB/s 


In [4]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset, ConcatDataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast, get_linear_schedule_with_warmup

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0)

fp16

In [5]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

     |████████████████████████████████| 51kB 3.4MB/s 


# Model... Train

## Load Model and Tokenizer

In [6]:
model_name = "hfl/chinese-roberta-wwm-ext-large"
# model_name = "ckiplab/bert-base-chinese" 
# model_name = "bert-base-chinese"
# model_name = "hfl/chinese-roberta-wwm-ext"

#my trained model
# model_name = "/content/gdrive/MyDrive/HW7/saved_model/model"

model = BertForQuestionAnswering.from_pretrained(model_name).to(device)

tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext-large were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl

## Read Data

In [7]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("/content/gdrive/MyDrive/HW7/hw7_train.json")
dev_questions, dev_paragraphs = read_data("/content/gdrive/MyDrive/HW7/hw7_dev.json")
test_questions, test_paragraphs = read_data("/content/gdrive/MyDrive/HW7/hw7_test.json")

## Tokenize Data

In [8]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

Token indices sequence length is longer than the specified maximum sequence length for this model (570 > 512). Running this sequence through the model will result in indexing errors


## Dataset and Dataloader
preprocessing

In [9]:

class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 40
        self.max_paragraph_len = 184
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 60

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            '''midddd'''
            mid = (answer_start_token + answer_end_token) // 2 + random.randint(-100,100)
            paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len
            
            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("train", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 16
data_set = train_set+dev_set
# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

train_loader = DataLoader(data_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)

## Function for Evaluation
postprocessing

In [10]:
def evaluate(data, output, para):

    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
          
            # prevent empty answer
            if start_index > end_index:
              continue
            else:  
              max_prob = prob
              # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
              answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
              # print(para[start_index : end_index + 1])
            
    # postprocessing
    if answer[0] == '《' and answer[-1] != '》':
      answer = answer + '》'
    if answer[0] != '《' and answer[-1] == '》':
      answer =  '《' + answer
    if answer[0] == '「' and answer[-1] != '」':
      answer = answer + '」'
    if answer[0] != '「' and answer[-1] == '」':
      answer =  '「' + answer

    answer = answer.replace(' ','')
    

    #fix unk sep
    flag = 0
    length = len(answer)
    if "[SEP]" in answer :
      answer = answer.replace('[SEP]','@')
      flag = 1
    if "[UNK]" in answer :
      answer = answer.replace('[UNK]','@')
      flag = 1
      
    if flag  == 1:
      at = -1
      for i,a in enumerate(answer):
          if a=='@':
              at = i
          
      if at == length-1:
          answer = answer[:at]
          start = para.find(answer)
          answer = para[start:start+length]
      elif at == 0:
          answer = answer.replace("@",'') 
          start = para.find(answer)
          answer = para[start-1:start+length-1]
      else:
          answer = answer[:at]
          start = para.find(answer)
          answer = para[start:start+length]

    

    return answer

## Training
learning rate ref:

https://blog.csdn.net/Xiao_CangTian/article/details/109269555

https://stackoverflow.com/questions/60120043/optimizer-and-scheduler-for-bert-fine-tuning

In [11]:
num_epoch = 1 
validation = True
logging_step = 100
learning_rate = 1e-4
optimizer = AdamW(model.parameters(), lr=learning_rate)


total_steps = len(train_loader) * num_epoch
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 20, num_training_steps = total_steps)


if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

model.train()

print("Start Training ...")


for epoch in range(num_epoch):
    
    step = 1
    train_loss = train_acc = 0
    
    for data in tqdm(train_loader):

        	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss
        
        if fp16_training:
            accelerator.backward(output.loss)
        else:
            output.loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()

        # scheduler.step()
        step += 1

        optimizer.param_groups[0]["lr"] -= learning_rate / total_steps
        
        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            # print(optimizer.param_groups[0]["lr"])
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0

    # if validation:
    #     print("Evaluating Dev Set ...")
    #     model.eval()
    #     with torch.no_grad():
    #         dev_acc = 0
    #         for i, data in enumerate(tqdm(dev_loader)):
    #             output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
    #                    attention_mask=data[2].squeeze(dim=0).to(device))
                
    #             # prediction is correct only if answer text exactly matches
    #             dev_acc += evaluate(data, output) == dev_questions[i]["answer_text"]
    #         print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
    #     model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "/content/gdrive/MyDrive/HW7/saved_model/model" 
model.save_pretrained(model_save_dir)

Start Training ...


Epoch 1 | Step 100 | loss = 1.623, acc = 0.502
Epoch 1 | Step 200 | loss = 0.988, acc = 0.671
Epoch 1 | Step 300 | loss = 1.038, acc = 0.663
Epoch 1 | Step 400 | loss = 0.994, acc = 0.648
Epoch 1 | Step 500 | loss = 0.978, acc = 0.660
Epoch 1 | Step 600 | loss = 0.875, acc = 0.676
Epoch 1 | Step 700 | loss = 0.996, acc = 0.654
Epoch 1 | Step 800 | loss = 1.019, acc = 0.662
Epoch 1 | Step 900 | loss = 0.857, acc = 0.692
Epoch 1 | Step 1000 | loss = 0.830, acc = 0.721
Epoch 1 | Step 1100 | loss = 0.822, acc = 0.694
Epoch 1 | Step 1200 | loss = 0.753, acc = 0.721
Epoch 1 | Step 1300 | loss = 0.767, acc = 0.721
Epoch 1 | Step 1400 | loss = 0.705, acc = 0.743
Epoch 1 | Step 1500 | loss = 0.704, acc = 0.734
Epoch 1 | Step 1600 | loss = 0.661, acc = 0.743
Epoch 1 | Step 1700 | loss = 0.610, acc = 0.754
Epoch 1 | Step 1800 | loss = 0.622, acc = 0.756
Epoch 1 | Step 1900 | loss = 0.598, acc = 0.774

Saving Model ...


# evl

In [ ]:
print("Evaluating Dev Set ...")
model.eval()
with torch.no_grad():
    dev_acc = 0
    for i, data in enumerate(tqdm(dev_loader)):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                attention_mask=data[2].squeeze(dim=0).to(device))
        
        para = dev_paragraphs[dev_questions[i]['paragraph_id']]

        answer = evaluate(data, output, para)
        print(answer, dev_questions[i]["answer_text"])
        dev_acc += answer == dev_questions[i]["answer_text"]
    print(f"acc = {dev_acc / len(dev_loader):.3f}")

Evaluating Dev Set ...


杜拜知識村 杜拜知識村
3500萬美元 3500萬美元
《胭脂扣》 《胭脂扣》
臺東市 臺東市
顏延之 顏延之
1985年 1985年
長江流域 長江流域
1920年 1920年
賓夕法尼亞州 賓夕法尼亞州
賓夕法尼亞州 賓夕法尼亞州
商稅 商稅
校園核能研究反應爐 校園核能研究反應爐
私立研究型大學 私立研究型大學
《涌幢小品·總督總兵》 《涌幢小品·總督總兵》
2017年 2017年
勸誡性長詩 勸誡性長詩
1951年 1951年
台灣海峽 台灣海峽
光緒31年 光緒31年
經濟力 經濟力
9日 9日
紐澤西美洲人 紐澤西美洲人
昭和四十九年 昭和四十九年
1922年 1922年
1793年 1793年
烏克蘭 英國
元朝 元朝
西夏 西夏
電影 電影
王莽 王莽
心理危機干預熱線 心理危機干預熱線
冉閔 冉閔
因周王是燕王同母兄弟，而朱允炆怕他與燕王呵成一氣，於是決 怕他與燕王呵成一氣
草書 草書
1984年 1984年
內蒙古 內蒙古
紅土 法網
鑑於新教徒對聖經正典的質疑 新教徒對聖經正典的質疑
不確定性 不確定性
香港 新加坡
蝦夷地 「蝦夷地」
蘇門答臘島 蘇門答臘島
加拿大 加拿大
公元前一萬年前 公元前一萬年前
丹麥人 丹麥人
索馬利亞 索馬利亞
位於天樂里 殯儀館
消費者 消費者
機器人系列 機器人系列
查理斯·萊爾 查理斯·萊爾
九龍東 九龍東
九龍塘省善真堂 九龍塘省善真堂
一個月 一個月
傍晚時份 傍晚時份
1909年 1909年
《德意志民主共和國憲法》 1949年
肝臟 肝臟
排灣族 世襲
科尼亞 羅姆蘇丹國首都科尼亞
俄語 俄語
《大公報》 《大公報》
網球 網球
波蘭 波蘭
胚芽米 白米
國民投票選舉 比例代表制
降雨的不均衡 降雨的不均衡
公共衛生問題 公共衛生問題
闢為公園，正式對民眾開放 闢為公園，正式對民眾開放
《周禮·考工記》 周禮·考工記
1971年4月 1971年4月
2015年 2015年
阿拉巴馬州 阿拉巴馬州


KeyboardInterrupt: ignored

In [12]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for i,data in enumerate(tqdm(test_loader)):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        para = test_paragraphs[test_questions[i]['paragraph_id']]
        ans = evaluate(data, output, para)
        print(ans)
        result.append(ans)

result_file = "result.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


白色羽毛
《諸羅縣志》
袁晁
柏林陸軍學院
一局特殊的決勝局
聶耳
1884年
西漢的版圖向西擴張
英國
醜化對手的作用
周星馳
三中老
無障礙環境
anticipation
2004年
硫化氫
華萊士
外面的部份溫度會比內部核心的低
經濟
100公里

荷蘭
硫磺
塞爾維亞反對黨
1910年代
外交活動
拉丁文
猶太基督徒
1964年
霍克菲爾德
被迫外遷
《薩蒂里卡》
灣仔
鄭和
形容一種無限制的、自私、而野蠻的競爭情況
1968年
新北市
小白宮
沒有
南人
宋端宗
四千名
46億年
1070年
《國民養老保險法》
水上報告廳
1929年
航天技術
1932年
大型購物中心MegaBox
康熙帝
自強學堂
1960年代
刺殺
紐約市
基督教
預覽阿爾法版本
100萬
新北市
佔領行動
路易十五
中國國民黨
中生代侏羅紀
阿拉貢國王卡洛斯三世
熱帶雨林氣候
意圖完全擺脫漢朝的控制
蔣中正
東漢
前三點起落架
弘治十二年
亞洲
1996年
德意志聯邦共和國
氦
西班牙人入侵納瓦爾
交通運輸部
高麗大學校主樓
朱少文
羽柴秀吉
俾斯麥
金泳三
1958年
「靈魂」
科龍港
蓋爾人
四次重差觀測術
西漢元封二年
隋滅南陳
佛教
1861年
豐臣秀吉
三吳地區
英國
大躍進運動
國立武昌高等師範學校
蘇利南
1934年
檸檬酸咖啡因
1913年
1949年
大象
1915年
1000年
有絲分裂
德國
高盧帝國
仁祖
十個
佛學論書
臺中高農
布倫瑞克
三天
1944年
安娜·德·薩伏伊皇后
咸豐帝
哈林河
孟加拉
男中音
中原
60
威廉·洛厄爾·普特南
中華人民共和國主席
德國
陳寅恪
刺殺
喃字的複雜及未標準化
二千九百公尺
1939年
害怕德軍在東線戰線上沒有足夠兵力應付俄國
福建林家
用來運貨
公民權
西人黨
2013年
臺北
查莫洛人
杭縣
蘇聯國旗
38%
1941年
多腔磁控管雷達技術
點石齋畫報
南越地區
1988年
上吊方式
吉米卡特
沈括
《布告各省公啟》
阿拉里克
德蘇戰爭
髒層
工業國家
1949年
《最終解決德國問題條約》
清光緒十七年
「雕像」
防止中國共產黨滲透
65,000平方公里
乙醚
唐太宗
卡爾梅克人
1969年11月12日
63位
副大法官
古希臘時代
盧安達語
自政府填海造地
蔣中正
農民
網景通訊公司
9月1日
1999年


In [ ]:
raise

# Ensemble

In [ ]:
my_model_names = ["/content/gdrive/MyDrive/HW7/saved_model/model1",
                  "/content/gdrive/MyDrive/HW7/saved_model/model2",
                  "/content/gdrive/MyDrive/HW7/saved_model/model3",
                  "/content/gdrive/MyDrive/HW7/saved_model/model4",
                  "/content/gdrive/MyDrive/HW7/saved_model/model5",
                  "/content/gdrive/MyDrive/HW7/saved_model/model6",
                  "/content/gdrive/MyDrive/HW7/saved_model/model7",
                  # "/content/gdrive/MyDrive/HW7/saved_model/model8",
                  ]


models = [BertForQuestionAnswering.from_pretrained(model_name).to(device) for model_name in my_model_names]

tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")

In [ ]:
def ensamble_evaluate(data, outputs):

    answer = ''
    glo_max_prob = float('-inf')
    num_of_windows = data[0].shape[1]

    child_answers = []
    for i, output in enumerate(outputs):
      
      loc_max_prob = float('-inf')

      for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        if prob > loc_max_prob:
            if start_index > end_index:
              continue
            else:
              loc_max_prob = prob
              child_answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])

        if prob > glo_max_prob:
            if start_index > end_index:
              continue
            else:  
              glo_max_prob = prob
              master_answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])

      child_answers.append(child_answer)
    
    
    count = 0
    for child_answer in child_answers:
      if master_answer == child_answer:
        count+=1 
      else:
        continue
    
    if count >= 3:
      answer = master_answer
    else:
      answer = child_answers[0]
      


    # postprocessing
    if answer[0] == '《' and answer[-1] != '》':
      answer = answer + '》'
    if answer[0] != '《' and answer[-1] == '》':
      answer =  '《' + answer
    if answer[0] == '「' and answer[-1] != '」':
      answer = answer + '」'
    if answer[0] != '「' and answer[-1] == '」':
      answer =  '「' + answer

    print(f'child_answers: {child_answers}')
    print(f'master_answer: {master_answer}')
    print(f'answer: {answer}')
    print()
    
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer.replace(' ','')

In [ ]:
print("Evaluating Dev Set with all models...")



with torch.no_grad():
    dev_acc = 0
    for i, data in enumerate(tqdm(dev_loader)):
        outputs = []
        for model in models:
            model.eval()
            output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                attention_mask=data[2].squeeze(dim=0).to(device))
            outputs.append(output)   
            
        # prediction is correct only if answer text exactly matches
        dev_acc += ensamble_evaluate(data, outputs) == dev_questions[i]["answer_text"]
    print(f"{dev_acc / len(dev_loader):.3f}")

## Testing

In [ ]:
print("Evaluating Test Set ...")

result = []
result_file = "result.csv"

with torch.no_grad():
    for data in tqdm(test_loader):
      outputs = []
      for model in models:
          model.eval()
          output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
              attention_mask=data[2].squeeze(dim=0).to(device))
          outputs.append(output)
      result.append(ensamble_evaluate(data, outputs))


with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")